# Executing Scripts
## Introduction

This notebook is about creating command line interface (CLI) programs. The topic is not covered by the book but there are good tutorials available online, such as [Real Python's tutorial on argparse](https://realpython.com/command-line-interfaces-python-argparse/).

You can find more information about creating CLI programs with the `argparse` library in the Python documentation: [Argparse Tutorial](https://docs.python.org/3/howto/argparse.html).

There are other great libraries when working with command line arguments, such as [Click](https://click.palletsprojects.com), [Fire](https://github.com/google/python-fire) and [Plumbum](https://plumbum.readthedocs.io/en/latest/cli.html).

## Summary

The `argparse` module lets you parse and store arguments passed via the console:

```python
import argparse
parser = argparse.ArgumentParser(description='Does something')
arguments = parser.parse_args()
```

Add positional arguments with `add_argument`:

```python
parser.add_argument('verbosity')
```

Add optional arguments by prefixing them with hyphens:

```python
parser.add_argument('-v', '--verbosity', help='increase output verbosity')
```

To add optional boolean parameters, use `action='store_true'` or `action='store_false'`. The full list of available actions is [documented in the Python docs](https://docs.python.org/3/library/argparse.html#action).

```python
parser.add_argument('-v', '--verbose', help='increase output verbosity', action='store_true')
```

To avoid executing some code in case your file gets imported by another file, use the magic variable `__name__`:

```python
import argparse

def main(name):
    print('Hello, %s!' % name)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description = 'Say hello')
    parser.add_argument('name', help='Your name')
    args = parser.parse_args()
    main(args.name)
```

To write to `stderr`, use `sys.stderr` (or use logging):

```python
sys.stderr.write('Some error occured')
print('Some error occured', file=sys.stderr)
```

To read environment variables, use `os.environ` which behaves like a dictionary:

```python
os.environ.get('ENV_NAME')
```



## Exercises

### Exercise 1: Creating a Parser
Create a parser which takes two positional arguments `first_name` and `last_name` and two optional arguments `salutation` (defaults to an empty string) and `underage` (boolean, set to `True` if provided). Make sure the cell executes without throwing an error.

In [ ]:
import argparse

parser = argparse.ArgumentParser()
# todo: add arguments

arguments = parser.parse_args(["John", "Doe"])
assert vars(arguments) == {
    "first_name": "John",
    "last_name": "Doe",
    "salutation": "",
    "underage": False,
}

arguments = parser.parse_args(["John", "Doe", "--salutation", "Sir"])
assert vars(arguments) == {
    "first_name": "John",
    "last_name": "Doe",
    "salutation": "Sir",
    "underage": False,
}

arguments = parser.parse_args(["Jane", "Doe", "--underage"])
assert vars(arguments) == {
    "first_name": "Jane",
    "last_name": "Doe",
    "salutation": "",
    "underage": True,
}

### Exercise 2: Drink-Generator as CLI-Application

Remember the drink generator you wrote in one of the earlier labs? Write a new version of it, but this time as a command-line application! Requirements:

* `-l` / `--list` lists all possible drinks that are available, no arguments needed.
* `-d` / `--drink` followed by `drink_name` as argument prints all the required ingredients for a drink.
* `-i` / `--ingredient` followed by three ingredients prints all possible drinks based on the argument. The ingredients argument is provided as one string, e.g. `"ice, gin tonic, water"`.
* The three arguments are mutually exclusive, if you enter for example `-l`, it is not allowed to enter `-d` or `-i` or any other combination. Hint: Look up `mutually exclusive group` in the documentation on argparse.

To get you started, the core code of the drink generator is provided.

Expected output:
```bash
$ python drink-generator-cli.py -l       

These drinks are available:
* caipirinha
* mojito
* gin tonic
* vodka martini

$ python drink-generator-cli.py -d "gin tonic"

Ingredients for a gin tonic:
gin, tonic water, ice

$ python drink-generator-cli.py -d nodrink

nodrink does not exist.

$ python drink-generator-cli.py -i "ice, tonic water, gin"

You can make gin tonic, how tasty!

$ python drink-generator-cli.py -i "ice, not, gin"  

No suitable drink found :-( Just mix everything together and you're fine!

$ python drink-generator-cli.py -l -i "ice, tonic water, gin"

usage: drink-generator-cli.py [-h] [-l | -d DRINK | -i INGREDIENTS]
drink-generator-cli.py: error: argument -i/--ingredients: not allowed with argument -l/--list

```

In [ ]:
%%writefile drink-generator.py
import argparse


def find_drink(available_ingredients, drinks):

    possible_drinks = []
    set_of_available_ingredients = set(available_ingredients)
    for drink, ingredients in drinks.items():
        if set_of_available_ingredients.issubset(ingredients):
            possible_drinks.append(drink)
        if possible_drinks:
            return possible_drinks
    return None


def main(list_of_drinks, drink, ingredients):
    drinks = {
        "caipirinha": ("cachaca", "sugar", "lime"),
        "mojito": ("white rum", "sugar cane juice", "lime juice", "soda water", "mint"),
        "gin tonic": ("gin", "tonic water", "ice"),
        "vodka martini": ("vodka", "vermouth", "ice", "olives"),
    }

    # todo: parse arguments and return correct output


if __name__ == "__main__":
    # todo: Create the parser and run the main function

In [ ]:
%run drink-generator.py -h

### Exercise 3: Converting Date/Times
Create a script which parses a date and time, converts it to given timezone and returns it in ISO 8601 format (UTC). You can use `dateutil` and `pytz` to make things a bit easier!

In [ ]:
!pip install python-dateutil pytz

In [ ]:
%%writefile convert-date.py
# todo: your solution here

Calling help on your command should return something like this:

```bash
usage: convert_date.py [-h] [-t TIMEZONE] datetime

Convert a given date to ISO 8601 format

positional arguments:
  datetime              Date/time to be converted

optional arguments:
  -h, --help            show this help message and exit
  -t TIMEZONE, --timezone TIMEZONE
                        Timezone of the given datetime
```

In [ ]:
%run convert-date.py -h

Calling your command should return:

- `convert-date.py '31.12.2018'`: `2018-12-31T00:00:00+00:00`
- `convert-date.py '31.12.2018 12:00'`: `2018-12-31T12:00:00+00:00`
- `convert-date.py '2018/1/13 11:05pm' --timezone 'US/Pacific'`: `2018-01-14T07:05:00+00:00`

In [ ]:
%run convert-date.py '31.12.2018'

In [ ]:
%run convert-date.py '31.12.2018 12:00'

In [ ]:
%run convert-date.py '2018/1/13 11:05pm' --timezone 'US/Pacific'